In [ ]:
# default_exp ppdd

# pico_pi_controller.ppdd
> System daemon

In [ ]:
# hide
# from nbdev.showdoc import *

In [ ]:
# export

from sys import byteorder
from os import getloadavg
from platform import node
from uptime import uptime
import threading, _thread
import argparse, logging
import pigpio, atexit
import datetime, time
import uuid, socketserver
import queue
from CircuitPython_pico_pi_common.codes import *

logger = logging.getLogger()
logging.basicConfig(level = logging.DEBUG)

In [ ]:
# export

tcp_address = ('127.0.0.1', 16164)
hostname = bytearray(node(), "utf-8")

def log_txn(fname, message, msg=None, i2c_addr=None):
        """Wrapper for logger."""
        id_str = ID_CODE.decode()
        hex_addr = ''
        if i2c_addr:
            hex_addr = str(hex(i2c_addr))
        i2c_str = '|'
        logger.info('%-4s %-47s %s' % (id_str, fname+': '+hex_addr+message+str(msg or ''), i2c_str))

class PwrLedFlicker(threading.Thread):
    def __init__(self, duration):
        threading.Thread.__init__(self)
        self.duration = duration
        
    def run(self):
        fname='run'
        try:
            with open("/sys/class/leds/led1/brightness", "w") as sys_pwr_led:
                for x in range(self.duration * 2):
                    sys_pwr_led.write("0")
                    sys_pwr_led.flush()
                    time.sleep(0.1)
                    sys_pwr_led.write("1")
                    sys_pwr_led.flush()
                    time.sleep(0.4)
            sys_pwr_led.close()
        except PermissionError:
            log_txn(fname,"Must run as root.")
            _thread.interrupt_main()
            
class PPCcHandler(socketserver.BaseRequestHandler):
    def handle(self):
        """Handle commands from ppcc; some processed iternally, some sent to PPC"""
        fname='handle'
        global cmd_queue, awt_queue, cfm_queue
        #log_txn(fname,"recvd request from {}".format(self.client_address[0]),
        #        " on {}".format(threading.current_thread().name))
        cmd_len = bytearray(1)
        cmd_len = int.from_bytes( self.request.recv(1), byteorder=byteorder)
        
        #log_txn(fname,"recvd command from {}, len: ".format(self.client_address[0]),str(cmd_len)+" bytes:")
        
        command = bytearray(cmd_len)
        command = self.request.recv(cmd_len)
        
        cmd_code, i2c_addr, cmdargs, cmd_uid, valid_status = parse_cmd(command)
        
        if valid_status:
        
            log_txn(fname,'',
                str(hex(i2c_addr))+' '+CMD_NAME[cmd_code]+' 0'+str(cmdargs)[3:-1]+"  UID "+str(hex(int.from_bytes(cmd_uid, byteorder)))[2:])
        
            cmd_queue.put(command)
            log_txn(fname,"command queue size:", cmd_queue.qsize() )
            # blocking, wait for confirmation of i2c_event_handler acting on command
            # todo: implement a timeout if confirm never appears in awt_queue
            confirm = bytearray(0)
            while confirm != command:
                log_txn(fname,"awaitng confirmation...")
                confirm = awt_queue.get(block=True,timeout=2)
                # put it back if it's not the confirm we're looking for, or if it's only a 
                # partial confirm, then log it & delete it.
                if confirm != command:
                # todo: handle partial confirms (only one 2c_addr from 0xFF)
                    awt_queue.put(confirm)
                log_txn(fname,"awaitng queue size:", awt_queue.qsize() )
            if confirm == command:
                cfm_queue.put(command)
                num_bytes_sent = self.request.send(command)
                log_txn(fname,"confirm queue size:", cfm_queue.qsize() )
        else:
            pass #handle error
        self.request.close()

@atexit.register
def goodbye():
    """Cancel pigpio event handler, close I2C peripheral & connection to pigpio"""
    global e, pi, ppdd_server
    fname='goodbye'
    if 'e' in globals():
        try:
            e.cancel()
            log_txn(fname,"bsc_i2c event handler cleanup completed.")
        except:
            log_txn(fname,"bsc_i2c error on event handler cleanup.")
    if 'pi' in globals():
        try:
            pi.stop()
            log_txn(fname,"pigpio cleanup completed.")
        except:
            log_txn(fname,"pigpio error on cleanup.")
    if 'ppdd_server' in globals():
        try:
            ppdd_server.server_close()
            log_txn(fname,"TCP serversocket cleanup completed.")
        except:
            log_txn(fname,"TCP serversocket error on cleanup.")
    log_txn(fname,"Exiting.")

def i2c_event_handler(id, tick):
    """Handle register probes."""
    fname='i2c_event_handler'
    global pi, i2c_addr, bosmang, flicker, cmd_queue, cfm_queue

    def reg_idf():
        fname='reg_idf'
        try:
            mcu_uid = d[1:8]
        except:
            mcu_uid = bytearray(8)
        log_txn(fname, "recvd identity: ", mcu_uid.decode())
        log_txn(fname, "sendn identity: ", ID_CODE.decode())
        s, b, d = pi.bsc_i2c(i2c_addr, ID_CODE)

    def reg_bos():
        fname='reg_bos'
        log_txn(fname, "sending bosmang status: ", str(bool(int.from_bytes(bosmang, byteorder=byteorder))))
        s, b, d = pi.bsc_i2c(i2c_addr, bosmang)

    def reg_tim():
        fname='reg_tim'
        dt = datetime.datetime.now()
        # send local time converted from UTC of system time
        dt = dt.replace(
            tzinfo=datetime.timezone(datetime.timedelta(0), "UTC") )  
        ts = dt.timestamp()
        log_txn(fname, "sending time.time(): ", int(ts) )
        s, b, d = pi.bsc_i2c(i2c_addr, bytearray(int(ts).to_bytes(4, byteorder)))

    def reg_cmd():
        fname='reg_cmd'
        global cmd_queue, awt_queue
        #log_txn(fname,"checking command queue")
        if cmd_queue.qsize() > 0:
            command=cmd_queue.get()
            if command[0] in CMD_NAME.keys(): 
                #log_txn(fname,"sending queud command: ", command )
                s, b, d = pi.bsc_i2c(i2c_addr, command)
                # place the command in the queue for awaiting confirmation
                awt_queue.put(command)
            else:
                log_txn(fname,"invalid command found in queue: ", command )
        else:
            s, b, d = pi.bsc_i2c(i2c_addr, bytearray([0]))
            #log_txn(fname,"no command")
            

    def reg_cfm():
        fname='cfm_cmd'
        global awt_queue 
        i2c_addr = d[0] # address of the ppd acted upon by the command
        if d[1] in CMD_INT:
            # add PPDevice CFM received to awt_queue (awaiting ppdd confirm); it may be
            # only 1 of many i2c_addrs. logic in handle.
            awt_queue.add(d[1:]) # CFM includes echo of entire command
            log_txn(fname,str(hex(i2c_addr))+" recvd confirm: ", CMD_NAME[d[0]])
            
    def reg_hos():
        fname='reg_hos'
        log_txn(fname,
            "sending hostname: ", hostname.decode())
        s, b, d = pi.bsc_i2c(i2c_addr, bytes([len(hostname)]) + hostname)

    def reg_lod():
        fname='reg_lod'
        load = bytearray("{:04.2f}".format(getloadavg()[0]), "utf-8")
        log_txn(fname, "sending loadavg: ", load.decode())
        s, b, d = pi.bsc_i2c(i2c_addr, load)

    def reg_upt():
        fname='reg_upt'
        # so the PPC can pause and wait for the long-ish uptime fetch
        s, b, d = pi.bsc_i2c(i2c_addr, bytearray(1) )
        uptimeba = bytearray(int(uptime()).to_bytes(4, byteorder))
        log_txn(fname, "sending uptime: ", int.from_bytes(bytes(uptimeba), byteorder)
        )
        s, b, d = pi.bsc_i2c(i2c_addr, uptimeba)

    def reg_tzn():
        fname='reg_tzn'
        log_txn(fname, "sending time.timezone(): ", time.timezone, )
        s, b, d = pi.bsc_i2c(i2c_addr, bytearray(time.timezone.to_bytes(3, byteorder)))

    def reg_flk():
        fname='reg_flk'
        nonlocal d
        log_txn(fname, "Flickering PWR LED for "+str(d[1])+" seconds.")
        flicker = PwrLedFlicker(duration=d[1])
        flicker.start()
        # if FLK was from a command, d will hold that command. echo it back to confirm execution.
        if len(d) > REG_VAL_LEN['FLK']:
            if d[REG_VAL_LEN['FLK']+1] in CMD_INT:
                s, b, d = pi.bsc_i2c(i2c_addr, d[REG_VAL_LEN['FLK']:] )

    def reg_clr():
        fname='reg_clr'
        #log_txn(fname, "CLR recieved, FIFO buffer to be emptied.")
        s, b, d = pi.bsc_i2c(i2c_addr, bytearray(0) )

    s, b, d = pi.bsc_i2c(i2c_addr)
    if b:
        #log_txn(fname, "Register probe recvd, status:",s)
        if d[0] in REG_NAME.keys():
            reg_hndlr = locals()['reg_'+REG_NAME[d[0]].lower()]
            #log_txn(fname, ": reg_"+REG_NAME[d[0]].lower())
            reg_hndlr()
        else:
            log_txn(fname, ": unrecognized REG probed: ",d[0])


def main():
    fname='main'
    global cmd_queue, awt_queue, cfm_queue, i2c_addr, id_str, e, pi, ppdd_server, flicker, bosmang
    
    cmd_queue = queue.Queue() #command queue (commands recvd from ppcc, outgoing to PPC)
    awt_queue = queue.Queue() #awaiting confirmation of execution of command queue
    cfm_queue = queue.Queue() #confirmed execution of command queue
    
    # flicker power LED on startup, checking for root access
    flicker = PwrLedFlicker(duration=2)
    flicker.start()
    time.sleep(0.1)
    
    # setup pigpio, checking that it's running
    pi = pigpio.pi()
    if not pi.connected:
        log_txn(fname,"pigpiod not running.")
        exit()

    # handle command-line arguments
    parser = argparse.ArgumentParser(description='ppdd argument parser')
    parser.add_argument("-a", default='0x13', type=str, help="Default I2C address: 0x13")
    parser.add_argument("-b", action="store_true", help="Set bosmang status to True")
    
    args, unknown = parser.parse_known_args()
    
    if unknown:
        log_txn(fname,"unrecognized command line arguments: ",unknown)
    
    # didn't comment before. what does this do if no args.a?
    i2c_addr = int(args.a,0)
    
    if args.b:
        bosmang = bytearray(int(1).to_bytes(1, byteorder))
    else:
        bosmang = bytearray(int(0).to_bytes(1, byteorder))
        
    # setup socket server for incoming commands from ppcc
    try:
        ppdd_server = socketserver.ThreadingTCPServer(tcp_address, PPCcHandler)
        log_txn(fname,"Listening for ppcc commands on TCP port ", tcp_address[1])
    except OSError:
        log_txn(fname,"TCP Error. Socket in use?")
        exit()
    
    threading.Thread(target=ppdd_server.serve_forever).start()
        
    log_txn(fname,"Initializing I2C peripheral on address: ", hex(i2c_addr))
    log_txn(fname,"--> hostname: ", hostname.decode())
    log_txn(fname,"--> bosmang:  ", str(bool(int.from_bytes(bosmang, byteorder=byteorder))))

    # setup event handler for incoming I2C messages from PPController
    e = pi.event_callback(pigpio.EVENT_BSC, i2c_event_handler)
    pi.bsc_i2c(i2c_addr)

    while True:
        time.sleep(120)
        # heartbeat
        log_txn(fname,"koradewu ",datetime.datetime.now().isoformat())
        
if __name__ == '__main__':
    try:
        main()
    except KeyboardInterrupt as e:
        # handled by atexit
        #exit(str('Exiting.'))
        pass

INFO:root:ppdd run: Must run as root.                          |


In [ ]:
%tb

No traceback available to show.


In [ ]:
# hide
try:
    from IPython.display import display, Javascript
    display(Javascript('IPython.notebook.save_checkpoint();'))
    from time import sleep
    sleep(0.3)
    from nbdev.export import notebook2script
    notebook2script()
except ModuleNotFoundError:
    pass
    """CircuitPython kernel has no nbdev"""

<IPython.core.display.Javascript object>

Converted 00_core.ipynb.
Converted 50_ppdd.ipynb.
Converted 60_ppcc.ipynb.
Converted index.ipynb.
